# Touch-Data Reverse Engineering

In [1]:
# enable local imports
import sys, os
sys.path.append(os.path.abspath('../utils'))

# imports
import libipts

import hexdump
import ctypes

In [2]:
class IptsReport4ff(ctypes.Structure):      # report with a single counter
    _pack_ = True
    _fields_ = [
        ('counter', ctypes.c_uint32),       # some counter, unknown, might be first two bytes only
    ]

    def __str__(self):
        return f"IptsReport4ff {{ counter={self.counter} }}"

In [3]:
class IptsReport400(ctypes.Structure):
    _pack_ = True
    _fields_ = [
        ('unknown1', ctypes.c_uint16),          # something related to delta in timestamp?
        ('counter',  ctypes.c_uint16),
        ('timestamp', ctypes.c_uint32),         # timestamp in nanoseconds?
    ]

    def __str__(self):
        return f"IptsReport400 {{ unknown={self.unknown1}, counter={self.counter}, timestamp={self.timestamp} }}"

In [4]:
# Parser to extract touch data
class TouchParser(libipts.Parser):
    def __init__(self):
        super().__init__()

        self.pfn_pldf[0x0a] = self._parse_payload_frame_0a

        self.pfn_rprt[0x033] = self._parse_report_033
        self.pfn_rprt[0x400] = self._parse_report_400
        self.pfn_rprt[0x402] = self._parse_report_402
        self.pfn_rprt[0x404] = self._parse_report_404
        self.pfn_rprt[0x406] = self._parse_report_406
        self.pfn_rprt[0x407] = self._parse_report_407
        self.pfn_rprt[0x432] = self._parse_report_432
        self.pfn_rprt[0x4ff] = self._parse_report_4ff
        self.pfn_rprt[0x800] = self._parse_report_800
        self.pfn_rprt[0x803] = self._parse_report_803
        self.pfn_rprt[0x804] = self._parse_report_804
        self.pfn_rprt[0x806] = self._parse_report_806
        self.pfn_rprt[0x807] = self._parse_report_807
        self.pfn_rprt[0x812] = self._parse_report_812
        self.pfn_rprt[0x8ff] = self._parse_report_8ff

    def parse(self, data):
        super().parse(data, silent=False)
    
    def _parse_payload_frame_0a(self, header, data):        # unknown, unknown, 1 byte, exclusively zero
        self.pos += len(data)

    def _parse_report_400(self, header, data):              # unknown, touch, 8 bytes, some timestamp and counter?
        rprt = IptsReport400.from_buffer_copy(data)
        print(rprt)
        self.pos += len(data)

    def _parse_report_402(self, header, data):              # unknown, unknown, 8 bytes
        self.pos += len(data)

    def _parse_report_404(self, header, data):              # unknown, touch, 36 bytes, weird sequences inside
        self.pos += len(data)

    def _parse_report_406(self, header, data):              # unknown, touch, 4 bytes, zeros
        self.pos += len(data)

    def _parse_report_407(self, header, data):              # unknown, touch, 4 bytes, zeros for touch, second byte is one for touch + stylus?
        self.pos += len(data)

    def _parse_report_432(self, header, data):              # unknown, unknown, 8 or 12 bytes
        self.pos += len(data)
    
    def _parse_report_4ff(self, header, data):              # unknown, touch, 4 bytes,
        rprt = IptsReport4ff.from_buffer_copy(data)         #   counter (maybe only first two bytes?), occasional wrong values
        self.pos += len(data)

    def _parse_report_800(self, header, data):              # unknown, unknown, 8 bytes, same as 0x400?
        rprt = IptsReport400.from_buffer_copy(data)
        self.pos += len(data)

    def _parse_report_803(self, header, data):              # same as 0x403 data wise, semantics unknown
        rprt = libipts.IptsHeatmapDim.from_buffer_copy(data)
        self.pos += len(data)

    def _parse_report_804(self, header, data):              # unknown, unknown, 36 bytes, weird sequences inside, same as 0x404?
        self.pos += len(data)

    def _parse_report_806(self, header, data):              # unknown, unknown, 4 bytes, zeros, same as 0x406?
        self.pos += len(data)

    def _parse_report_807(self, header, data):              # unknown, unknown, 4 bytes, zeros
        self.pos += len(data)

    def _parse_report_812(self, header, data):              # unknown, unknown, 24 bytes,
        self.pos += len(data)

    def _parse_report_033(self, header, data):              # unknown, touch, 24 bytes, lots of zeros, 
        self.pos += len(data)                               #  occasional loosely correlated non-zero value in 2nd byte

    def _parse_report_8ff(self, header, data):              # unknown, unknown, 4 bytes, same as 0x4ff? similar value as 0x4ff
        self.pos += len(data)
    
    # Note: 0x8xx reports seem to be sent at/close to start

In [5]:
file = '../data/touch-sb2_13-normal-1.iptsraw'

with open(file, 'rb') as f:
    data = f.read()

heatmaps = TouchParser().parse(data)

IptsReport400 { unknown=8, counter=12475, timestamp=17774549 }
IptsReport400 { unknown=520, counter=12477, timestamp=17785979 }
IptsReport400 { unknown=520, counter=12478, timestamp=17795969 }
IptsReport400 { unknown=520, counter=12479, timestamp=17805962 }
IptsReport400 { unknown=520, counter=12480, timestamp=17815949 }
IptsReport400 { unknown=520, counter=12481, timestamp=17825942 }
IptsReport400 { unknown=520, counter=12482, timestamp=17835929 }
IptsReport400 { unknown=520, counter=12483, timestamp=17845924 }
IptsReport400 { unknown=520, counter=12484, timestamp=17855909 }
IptsReport400 { unknown=520, counter=12485, timestamp=17865902 }
IptsReport400 { unknown=520, counter=12486, timestamp=17875891 }
IptsReport400 { unknown=520, counter=12487, timestamp=17885882 }
IptsReport400 { unknown=520, counter=12488, timestamp=17895871 }
IptsReport400 { unknown=520, counter=12489, timestamp=17905862 }
IptsReport400 { unknown=520, counter=12490, timestamp=17915849 }
IptsReport400 { unknown=520